In this notebook, we're going to learn how to clean up inconsistent text entries.

Get our environment set up
The first thing we'll need to do is load in the libraries and dataset we'll be using.

In [7]:
 #modules we will use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

# read in all our data
pakistan = pd.read_csv("C:\\Users\\hp\\Desktop\\data handling\\database\\pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)

1) Examine another column
Write code below to take a look at all the unique values in the "Graduated from" column.

When I tried to read in the Pakistan_intellectual_capital.csvfile the first time, I got a character encoding error, so I'm going to quickly check out what the encoding should be...

In [8]:
# look at the first ten thousand bytes to guess the character encoding
with open("C:\\Users\\hp\\Desktop\\data handling\\database\\pakistan_intellectual_capital.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(100000))

# check what the character encoding might be
print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [10]:
# read in our dat
pakistan = pd.read_csv("C:\\Users\\hp\\Desktop\\data handling\\database\\pakistan_intellectual_capital.csv", 
                              encoding='utf-8')


Now we're ready to get started! You can, as always, take a moment here to look at the data and get familiar with it. :)

For this exercise, I'm interested in cleaning up the "Graduated from" column to make sure there's no data entry inconsistencies in it. We could go through and check each row by hand, of course, and hand-correct inconsistencies when we find them. There's a more efficient way to do this though!

In [11]:
# get all the unique values in the 'Graduated from' column
Graduated = pakistan['Graduated from'].unique()

# sort them alphabetically and then take a closer look
Graduated.sort()
Graduated

array([' Columbia University', ' Delft University of Technology',
       ' Iowa State University', ' University of Central Florida',
       ' University of Innsbruck',
       ' University of Texas at Arlington (UTA)', ' University of Turin',
       'Abasyn University', 'Abdul Wali Khan University, Mardan',
       'Abdus Salam School of Mathematical Sciences,GC University',
       'Agricultural University Peshawar', 'Allama Iqbal Open University',
       'Asian Institute of Technology', 'Aston University, Birmingham',
       'Australian National University, Caneberra', 'BUKC',
       'Bahauddin Zakariya University', 'Bahria University',
       'Bahria University,Islamabad',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'Barani Institute of Information Technology',
       'Beaconhouse National University', 'Beihang University',
       'Beijing Institute of Technology',
       'Beijing Institute of Technology Beijing',
       'Beiji

Convert every entry in the "Graduated from" column in the professors DataFrame to remove white spaces at the beginning and end of cells.

In [12]:
# remove trailing white spaces
pakistan['Graduated from'] = pakistan['Graduated from'].str.strip()

In [13]:
# get all the unique values in the 'City' column
Graduated = pakistan['Graduated from'].unique()

# sort them alphabetically and then take a closer look
Graduated.sort()
Graduated

array(['Abasyn University', 'Abdul Wali Khan University, Mardan',
       'Abdus Salam School of Mathematical Sciences,GC University',
       'Agricultural University Peshawar', 'Allama Iqbal Open University',
       'Asian Institute of Technology', 'Aston University, Birmingham',
       'Australian National University, Caneberra', 'BUKC',
       'Bahauddin Zakariya University', 'Bahria University',
       'Bahria University,Islamabad',
       'Balochistan University of Information Technology, Engineering and Management Sciences',
       'Barani Institute of Information Technology',
       'Beaconhouse National University', 'Beihang University',
       'Beijing Institute of Technology',
       'Beijing Institute of Technology Beijing',
       'Beijing University of Posts & Telecommunications',
       'Biztek Institute Of Business & Technology,Karachi',
       'Blekinge Institute of Technology', 'Boston University',
       'Brock University Canada', 'Brunel University',
       'CECOS Uni

) Continue working with countries
In the tutorial, we focused on cleaning up inconsistencies in the "Country" column. Run the code cell below to view the list of unique values that we ended with.

In [25]:
# get all the unique values in the 'City' column
countries = pakistan['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

In [37]:
# convert to lower case
pakistan['Country'] = pakistan['Country'].str.lower()
# remove trailing white spaces
pakistan['Country'] = pakistan['Country'].str.strip()

Take another look at the "Country" column and see if there's any more data cleaning we need to do.

It looks like 'usa' and 'usofa' should be the same country. Correct the "Country" column in the dataframe so that 'usofa' appears instead as 'usa'.

In [38]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("USA", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('usa', 100),
 ('usofa', 75),
 ('austria', 60),
 ('australia', 50),
 ('spain', 50),
 ('urbana', 44),
 ('uk', 40),
 ('malaysia', 36),
 ('pakistan', 36),
 ('portugal', 36)]

In [42]:
# function to replace rows in the provided column of the provided dataframe
# that match the provided string above the provided ratio with the provided string
def replace_matches_in_column(df, column, string_to_match, min_ratio = 75):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")


In [43]:
# use the function we just wrote to replace close matches to "USA" with "USofA"
replace_matches_in_column(df=professors, column='Country', string_to_match="usa")

All done!


In [44]:
# get all the unique values in the 'Country' column
countries = pakistan['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [45]:
# use the function we just wrote to replace close matches to "USA" with "USofA"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")

All done!


In [46]:
# get all the unique values in the 'Country' column
countries = pakistan['Country'].unique()

# sort them alphabetically and then take a closer look
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)